In [1]:
import json
import pandas as pd
import numpy as np
import tqdm
from math import sqrt, floor
import random
import networkx as nx
import pickle
from timeit import default_timer as timer

### json 파일 불러오기

In [4]:
data = []
with open('../Epinion/epinions_2.json', 'r', encoding='UTF-8') as f:
    for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))
item = pd.DataFrame(data)

In [5]:
temp = []
for index,value in item.iterrows():
    temp.append(json.loads(value[0].replace("'","\"")))

In [6]:
df = pd.DataFrame(temp)

In [7]:
print("참여하는 유저의 수 : ", len(df['user'].unique()))
print("참여하는 item의 수 : ", len(df['item'].unique()))
print("interaction의 수 : ", len(df))

참여하는 유저의 수 :  116260
참여하는 item의 수 :  41269
interaction의 수 :  188478


In [9]:
trust = pd.read_table('..\\데이터\\Epinion\\network_trust.txt', sep = ' ', header=None,names=['give', 'trust', 'take'])

In [10]:
trust.head()

,give,trust,take
0,lavenderfruit,trust,kristinafh
1,shippo225,trust,sallyg8
2,aryeh7000,trust,liteair
3,chiquita,trust,magenta321
4,jeffsullivan,trust,rpllingrock


In [11]:
trust_user = set()
trust_user.update(trust['give'].unique())
trust_user.update(trust['take'].unique())

In [12]:
print(len(trust_user))

84531


In [13]:
review_user = set(df['user'].unique())

In [14]:
print('review를 남기지 않은 user의 수 : ' ,len(trust_user-review_user))
print('link가 없는 user의 수 : ', len(review_user-trust_user))

review를 남기지 않은 user의 수 :  66916
link가 없는 user의 수 :  98645


In [15]:
user_set = set()
user_set.update(trust['give'].unique())
user_set.update(trust['take'].unique())
user_set.update(df['user'].unique())

In [16]:
df.columns

Index(['user', 'stars', 'time', 'item', 'review'], dtype='object')

## user에 index를 부여하고 저장

In [8]:
df = df.drop_duplicates(keep='first', subset=['user','item'])

In [30]:
print(len(df_))
print(len(df))

188432
188128


In [9]:
del_index = []
def delete_func(x):
    if len(x['item']) <= 1:
        del_index.extend(x.index)
start = timer()
df.groupby(['user']).apply(delete_func)
df = df.drop(del_index)
print("전처리 시간 : ", timer()-start)
print("User의 수 : ", len(df['user'].unique()))
print("Item의 수 : ", len(df['item'].unique()))

전처리 시간 :  7.053114973939955
User의 수 :  27453
Item의 수 :  37274


In [10]:
start = timer()
item_index = {}
user_index = {}
user_dict = {}
user_set = set()
item_set = set()
u_index = 0
i_index = 0
for index, value in df.iterrows():
    if value['user'] not in user_set:
        user_index[value['user']] = u_index
        user_dict[u_index] = value['user']
        user_set.add(value['user'])
        u_index += 1
    if value['item'] not in item_set:
        item_index[value['item']] = i_index
        item_set.add(value['item'])
        i_index += 1
print("총 전처리 시간 : ", timer()-start)

총 전처리 시간 :  14.688854024978355


In [11]:
df['user'] = df['user'].map(lambda x: user_index[x])
df['item'] = df['item'].map(lambda x: item_index[x])

## timestamp도 같이 저장

In [35]:
temp_list = []
for val in df.iterrows():
    temp_list.append([val[1][0], val[1][3], val[1][2]])

In [26]:
temp = pd.DataFrame(temp_list, columns= ['user', 'item', 'time'])

In [34]:
temp.to_csv('../Epinion/epinion_time.txt', header=True, sep = ' ', index=False)

In [45]:
start = timer()
train_dict = {}
test_dict = {}
def split(x):
    order = x.sort_values(by = ['time'], axis=0)
    train_length = floor(len(order)*0.8)
    train = order.iloc[0:train_length,:]
    test = order.iloc[train_length:,:]
    train_dict[x.name] = list(train['item'])
    test_dict[x.name] = list(test['item'])
temp_group = temp.groupby(by = ['user']).apply(split)
print("전처리 시간 : ",timer()-start)
train_set = train_dict
test_set = test_dict

## time sampling으로 전처리

In [36]:
inter_dict = {}
def interaction_func(x):
    inter_dict[x.name] = list(x['item'])
df.groupby(by=['user']).apply(interaction_func)

""


In [37]:
with open('../데이터/Epinion/total_data.txt', 'w') as output:
    for k, v in inter_dict.items():
        line = [k] + v
        output.write(" ".join(map(str,line)))
        output.write('\n')

In [38]:
train_set = {}
test_set = {}
with open('../데이터/Epinion/total_data.txt', 'r') as output:
    for l in output.readlines():
        l = l.strip('\n').split(' ')
        items = [int(i) for i in l[1:]]
        uid = int(l[0])
        train_length = floor(len(items)*0.8)
        train_set[uid] = random.sample(items, train_length)
        test_set[uid] = list(set(items) - set(train_set[uid]))
        if len(test_set[uid]) <=0:
            print("uid ", uid, "train set : ", train_set[uid], "total items: ", items)
    

## Train과 Test를 따로 저장

In [48]:
with open('../Epinion/train.txt', 'w') as output:
    for k, v in train_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')
with open('../Epinion/test.txt', 'w') as output:
    for k, v in test_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')

In [42]:
map_user_df = pd.DataFrame(zip(user_index.keys(),user_index.values()), columns= ['org_id', 'remap_id'])
map_user_df.to_csv('../데이터/Epinion/user_list.txt', sep = '\t', index =False)

In [43]:
map_df = pd.DataFrame(zip(item_index.keys(),item_index.values()), columns= ['org_id', 'remap_id'])
map_df.to_csv('../데이터/Epinion/item_list.txt', sep = '\t', index = False)